# TIPS-Treasury Arbitrage Replication (2010-2020)

This notebook replicates the TIPS–Treasury arbitrage strategy for the period 2010–2020 following the methodology of Fleckenstein et al. (2014). It also outlines how a trader could implement this strategy in practice.

## Methods

### Replication Strategy

1. **Data Sources**:
   - **TIPS Data:** Daily yield data for TIPS from U.S. Treasury sources.
   - **Inflation Swap Data:** Zero-coupon inflation swap rates from Bloomberg.
   - **Nominal Treasury Data:** Daily nominal Treasury yields from CRSP.

2. **Synthetic Yield Construction:**

   For a given maturity \(\tau\):

   \begin{equation}
   \text{Synthetic Yield}_{t,\tau} = s_{t,\tau} + f_{t,\tau}
   \end{equation}

   where \(s_{t,\tau}\) is the TIPS yield and \(f_{t,\tau}\) is the fixed rate from the inflation swap. The arbitrage spread is then:

   \begin{equation}
   \text{Spread}_{t,\tau} = (s_{t,\tau} + f_{t,\tau}) - y_{T,t,\tau}
   \end{equation}

   A positive spread indicates that nominal Treasuries are overpriced relative to the synthetic yield.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style="dark")

### Load Data

The following cell loads the TIPS-Treasury arbitrage data from a parquet file. This file is generated from `compute_tips_treasury.py`. This formulates a tidy set of the data.

- Columns starting with "real_": TIPS real yields for each tenor (e.g., real_cc2, real_cc5, real_cc10, real_cc20)
expressed in decimal form (e.g., 0.02 for 2%).
- Columns starting with "nom_": Computed nominal zero-coupon Treasury yields for each tenor
(e.g., nom_zc2, nom_zc5, nom_zc10, nom_zc20) expressed in basis points.
- Columns starting with "tips_": TIPS-implied risk-free rates (e.g., tips_treas_2_rf, tips_treas_5_rf,
tips_treas_10_rf, tips_treas_20_rf) expressed in basis points.
- Columns starting with "arb_": Arbitrage measures (e.g., arb_2, arb_5, arb_10, arb_20) representing the
difference between the TIPS-implied risk-free rate and the corresponding nominal yield (tips_treas_{t}_rf - nom_zc{t}).


In [ ]:
rf = pd.read_parquet("../_data/tips_treasury_implied_rf.parquet")
rf

Selected region

In [ ]:
test = pd.DataFrame(index=rf.index)
test = rf[['arb_2', 'arb_5', 'arb_10', 'arb_20']]
test['date'] = rf['date']
test.index = test['date']
test.drop('date', axis=1, inplace=True)
test

### Generate Summary Statistics

The cell below computes summary statistics using `generate_figures.py` for each TIPS-Treasury arbitrage series over the period 2010-2020.

In [ ]:
import statsmodels.api as sm
import numpy as np
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
import generate_figures

# Compute summary statistics for the period 2010-01-01 to 2020-02-28
summary_stats = generate_figures.generate_summary_statistics(test, '2010-01-01', '2020-02-28')
summary_stats

### Plot TIPS-Treasury Spreads

The following cell plots the arbitrage spreads for various maturities (2Y, 5Y, 10Y, 20Y) over the period 2010-2020.

In [ ]:
fig = generate_figures.plot_tips_treasury_spreads(test, '2010-01-01', '2020-02-28')
plt.show()

### Practical Implementation

A trader could use the following steps to exploit the arbitrage opportunity:

1. **Signal Identification:** Continuously monitor the arbitrage spread using updated TIPS, inflation swap, and Treasury data. A consistently positive spread signals an opportunity.

2. **Position Construction:**
   - **Long Position:** Purchase TIPS and enter into corresponding zero-coupon inflation swap contracts to convert the real cash flows into fixed nominal cash flows.
   - **Short Position:** Short the nominal Treasury bond (or use Treasury futures/repurchase agreements as a proxy).

3. **Cash Flow Matching:** Utilize Treasury STRIPS or additional derivatives to ensure that the cash flows from the long and short positions align, thereby locking in the arbitrage profit.

4. **Risk Management:** Monitor execution costs, margin requirements, and liquidity risks. Funding constraints and counterparty risks must also be managed.

This strategy, while theoretically risk-free, requires meticulous execution and risk management in practice.

## Conclusion

The analysis confirms that a positive TIPS-Treasury arbitrage spread persisted throughout 2010–2020. Although the magnitude of the mispricing is lower than during the earlier period covered by Fleckenstein et al. (2014), the spread remains statistically and economically significant. This persistence underscores the impact of market frictions and funding constraints on arbitrage opportunities. In practice, traders could exploit this anomaly by constructing synthetic nominal positions using TIPS, inflation swaps, and nominal Treasury bonds, while carefully managing execution and funding risks.